In [1]:
import tensorflow as tf
import csv
import os
import numpy as np
from tensorflow import keras as tfk
from keras import layers as tfkl
import sys
from data_creation import *
import importlib

# Import constants and catch any import errors
from link_to_constants import *
link_constants()
try:
    from constants import * # type: ignore
    print("Successfully imported constants.")
except ImportError as e:
    print(f"Error importing constants: {e}")

Current directory: c:\Users\marco\Documents\GitHub\Tesi\Codice_carla\final - Copy\neural_network
Parent directory: c:\Users\marco\Documents\GitHub\Tesi\Codice_carla\final - Copy
Data gen and processing directory: c:\Users\marco\Documents\GitHub\Tesi\Codice_carla\final - Copy\data_gen_and_processing
Successfully imported constants.


In [10]:
def load_points_grid_map(csv_file):
    """Load bounding box vertices from a CSV file."""
    points = []
    with open(csv_file, 'r') as file:
        reader = csv.reader(file)
        for row in reader:
            # Extract the 3D coordinates of the 8 bounding box vertices
            coordinates = [
                float(row[0]), float(row[1]), float(row[2])
                ]
            points.append(coordinates)
    np_points = np.array(points)
    return np_points

def generate_grid_map (grid_map_path):
    grid_map_files = [f for f in os.listdir(grid_map_path)]
    list_grid_maps = []

    for file in grid_map_files:
        complete_path = os.path.join(grid_map_path, file)
        print(f"Loading {file}...")
        points = load_points_grid_map(complete_path)

        # Recreate the grid map from positions array
        grid_map_recreate = np.full((Y_RANGE, X_RANGE), FLOOR_HEIGHT) # type: ignore

        # Fill the grid map with values from positions array
        for pos in points:
            col, row, height = pos
            grid_map_recreate[int(row), int(col)] = height

        list_grid_maps.append(grid_map_recreate)
    
    return list_grid_maps

def load_points_grid_map_BB (csv_file):
    """Load bounding box vertices from a CSV file."""
    points = []
    with open(csv_file, 'r') as file:
        reader = csv.reader(file)
        next(reader)  # Skip header
        
        for row in reader:
            # Extract the 3D coordinates of the 8 bounding box vertices
            coordinates = [ [ float(row[i]), float(row[i+1]), float(row[i+2]) ] for i in range(2, 12, 3)]
            points.append(coordinates)

    np_points = np.array(points)
    return np_points

def generate_grid_map_BB (grid_map_path):
    grid_map_files = [f for f in os.listdir(grid_map_path)]
    list_grid_maps = []

    for file in grid_map_files:
        complete_path = os.path.join(grid_map_path, file)
        print(f"Loading {file}...")
        points = load_points_grid_map_BB(complete_path)

        # Recreate the grid map from positions array
        grid_map_recreate = np.full((Y_RANGE, X_RANGE), FLOOR_HEIGHT) # type: ignore

        # Fill the grid map with values from positions array
        for i in range (len(points)):
            for j in range(4):
                col, row, height = points[i][j]
                grid_map_recreate[int(row), int(col)] = height

        list_grid_maps.append(grid_map_recreate)
    
    return list_grid_maps

def split_data(lidar_data, BB_data, size):
    # Split the dataset into a combined training and validation set, and a separate test set
    X_train_val, X_test, y_train_val, y_test = train_test_split(
        lidar_data, # Samples
        BB_data, # Labels
        test_size = size,
        random_state=SEED # type: ignore
    )
    return X_train_val, X_test, y_train_val, y_test

In [9]:
complete_grid_maps = []
complete_grid_maps_BB = []

# Load sensor1
grid_maps = generate_grid_map(LIDAR_1_GRID_DIRECTORY) # type: ignore
grid_maps_BB = generate_grid_map_BB(NEW_POSITIONS_LIDAR_1_GRID_DIRECTORY) # type: ignore

complete_grid_maps.append(grid_maps)
complete_grid_maps_BB.append(grid_maps_BB)

# Load sensor2
grid_maps = generate_grid_map(LIDAR_2_GRID_DIRECTORY) # type: ignore
grid_maps_BB = generate_grid_map_BB(NEW_POSITIONS_LIDAR_2_GRID_DIRECTORY) # type: ignore

complete_grid_maps.append(grid_maps)
complete_grid_maps_BB.append(grid_maps_BB)

# Load sensor3
grid_maps = generate_grid_map(LIDAR_3_GRID_DIRECTORY) # type: ignore
grid_maps_BB = generate_grid_map_BB(NEW_POSITIONS_LIDAR_3_GRID_DIRECTORY) # type: ignore

complete_grid_maps.append(grid_maps)
complete_grid_maps_BB.append(grid_maps_BB)

Loading 20250118_102725_853040_169399.csv...
Loading 20250118_102725_975855_169402.csv...
Loading 20250118_102726_071063_169404.csv...
Loading 20250118_102726_162930_169406.csv...
Loading 20250118_102726_243632_169408.csv...
Loading 20250118_102726_321813_169410.csv...
Loading 20250118_102726_399399_169412.csv...
Loading 20250118_102726_476882_169414.csv...
Loading 20250118_102726_558496_169416.csv...
Loading 20250118_102726_635325_169418.csv...
Loading 20250118_102726_713490_169420.csv...
Loading 20250118_102726_789580_169422.csv...
Loading 20250118_102726_868563_169424.csv...
Loading 20250118_102726_947714_169426.csv...
Loading 20250118_102727_022729_169428.csv...
Loading 20250118_102727_100899_169430.csv...
Loading 20250118_102727_181769_169432.csv...
Loading 20250118_102727_260511_169434.csv...
Loading 20250118_102727_348745_169436.csv...
Loading 20250118_102727_432632_169438.csv...
Loading 20250118_102727_522459_169440.csv...
Loading 20250118_102727_614992_169442.csv...
Loading 20

In [11]:
print(len(complete_grid_maps[0]), len(complete_grid_maps[1]), len(complete_grid_maps[2]))

284 367 317


In [12]:
print(len(complete_grid_maps_BB[0]), len(complete_grid_maps_BB[1]), len(complete_grid_maps_BB[2]))

284 367 317


In [13]:
# Concatenate the lists in complete_grid_maps along the first dimension
complete_grid_maps_np = np.concatenate(complete_grid_maps, axis=0)
print(complete_grid_maps_np.shape)

(968, 400, 400)


In [14]:
# Concatenate the lists in complete_grid_maps_BB along the first dimension
complete_grid_maps_BB_np = np.concatenate(complete_grid_maps_BB, axis=0)
print(complete_grid_maps_BB_np.shape)

(968, 400, 400)


In [16]:
 # Split the data
X_train_val, X_test, y_train_val, y_test = split_data(complete_grid_maps_np, complete_grid_maps_BB_np, TEST_SIZE) # type: ignore
X_train, X_val, y_train, y_val = split_data(X_train_val, y_train_val, len(X_test)) # Esure that val and test set have the same lenght

In [17]:
print(X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape)

(774, 400, 400) (97, 400, 400) (97, 400, 400) (774, 400, 400) (97, 400, 400) (97, 400, 400)


In [18]:
from sklearn.preprocessing import MinMaxScaler

# Initialize the scaler
scaler = MinMaxScaler()

# Normalize the data
X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_val = scaler.transform(X_val.reshape(-1, X_val.shape[-1])).reshape(X_val.shape)
X_test = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)

y_train = scaler.fit_transform(y_train.reshape(-1, y_train.shape[-1])).reshape(y_train.shape)
y_val = scaler.transform(y_val.reshape(-1, y_val.shape[-1])).reshape(y_val.shape)
y_test = scaler.transform(y_test.reshape(-1, y_test.shape[-1])).reshape(y_test.shape)

In [ ]:
shape_input = (X_RANGE,Y_RANGE,1) # type: ignore
    
# Input layer for the 2.5D grid map
grid_map_input = tfkl.Input(shape_input, name='grid_map_input')

# Encoder
x = tfkl.Conv2D(32, (3, 3), activation='relu', padding='same')(grid_map_input)
x = tfkl.MaxPooling2D((2, 2), padding='same')(x)
x = tfkl.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = tfkl.MaxPooling2D((2, 2), padding='same')(x)
x = tfkl.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
encoded = tfkl.MaxPooling2D((2, 2), padding='same')(x)

# Decoder
x = tfkl.Conv2D(128, (3, 3), activation='relu', padding='same')(encoded)
x = tfkl.UpSampling2D((2, 2))(x)
x = tfkl.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = tfkl.UpSampling2D((2, 2))(x)
x = tfkl.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = tfkl.UpSampling2D((2, 2))(x)
decoded = tfkl.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

# Create the model
model = tfkl.Model(inputs=grid_map_input, outputs=decoded)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Print the model summary
model.summary()